In [30]:
# %pip install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_classif
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('sampled_tbc_training_data.csv')

In [5]:
total_duplicates = data.duplicated().sum()
print(f'Total duplicate rows: {total_duplicates}')

Total duplicate rows: 0


In [6]:
n_rows = len(data)
n_exact_duplicates = n_rows - data.drop_duplicates().shape[0]
print("Exact duplicate rows:", n_exact_duplicates)

Exact duplicate rows: 0


In [7]:
rows_in_dup_groups = data[data.duplicated(keep=False)].shape[0]
print("Rows in duplicate groups (keep=False):", rows_in_dup_groups)

Rows in duplicate groups (keep=False): 0


In [8]:
missing_values = data.isnull().sum()
percentage_of_missing = (missing_values / len(data)) * 100
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing %': percentage_of_missing.round(2)
}).sort_values(by='Missing %', ascending=False)

print("Missing values per column:")
display(missing_summary)

Missing values per column:


,Missing Values,Missing %
Customer_ID,0,0.0
recency,0,0.0
frequency,0,0.0
value,0,0.0
age,0,0.0
...,...,...
ITADMILLNM,0,0.0
ITADSEPLUS,0,0.0
ITBCCORE,0,0.0
ITOYODRVN,0,0.0


In [9]:
print(data.isnull().values.any())   # True if any missing value exists

False


In [10]:
data[data.isnull().any(axis=1)].head(10)   # Show first 10 rows that contain at least one missing value

,Customer_ID,recency,frequency,value,age,units,Units_BLT,Units_FRM,Units_IND,Units_MBT,...,INITTOENT,INTWMT,INTWT3,IPIRFSTRK,ISIGNPRO,ITADMILLNM,ITADSEPLUS,ITBCCORE,ITOYODRVN,OREXCEPT


In [26]:
dtype_summary = pd.DataFrame(data.dtypes, columns=['Data Type'])
display(dtype_summary)

,Data Type
Customer_ID,int64
recency,float64
frequency,float64
value,float64
age,float64
...,...
ITADMILLNM,int64
ITADSEPLUS,int64
ITBCCORE,int64
ITOYODRVN,int64


In [12]:
data.dtypes.value_counts()

float64    203
int64       26
Name: count, dtype: int64

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 229 entries, Customer_ID to OREXCEPT
dtypes: float64(203), int64(26)
memory usage: 3.5 MB


In [14]:
data.columns.tolist()

['Customer_ID',
 'recency',
 'frequency',
 'value',
 'age',
 'units',
 'Units_BLT',
 'Units_FRM',
 'Units_IND',
 'Units_MBT',
 'Units_MRT',
 'Units_OTR',
 'Units_PASS',
 'Units_PSUV',
 'Units_RLT',
 'Units_SPC',
 'Units_TRLR',
 'Units_-',
 'Units_Customer-Billed',
 'Units_National Acct',
 'Units_ADVANCE',
 'Units_BFGOODRICH',
 'Units_BKT',
 'Units_BRIDGESTONE',
 'Units_CARLISLE',
 'Units_CONTINENTAL',
 'Units_COOPER',
 'Units_CORDOVAN',
 'Units_CROSSWIND',
 'Units_DEESTONE',
 'Units_DELTA',
 'Units_DORAL',
 'Units_DOUBLE COIN',
 'Units_DURATURN',
 'Units_DURO',
 'Units_EL DORADO',
 'Units_FIRESTONE',
 'Units_GENERAL',
 'Units_GLADIATOR',
 'Units_GOODYEAR',
 'Units_HANKOOK',
 'Units_HARVEST KING',
 'Units_LAUFENN',
 'Units_MALHOTRA',
 'Units_MASTERCRAFT',
 'Units_MAXXIS',
 'Units_MICHELIN',
 'Units_MULTI MILE',
 'Units_NANCO',
 'Units_NANKANG',
 'Units_NATIONAL',
 'Units_NEXEN',
 'Units_NITTO',
 'Units_NORTECH',
 'Units_PIRELLI',
 'Units_POWER KING',
 'Units_PRINX',
 'Units_SAILUN',
 'U

In [15]:
print(data.columns[:20])
print(data.columns[-20:])

Index(['Customer_ID', 'recency', 'frequency', 'value', 'age', 'units',
       'Units_BLT', 'Units_FRM', 'Units_IND', 'Units_MBT', 'Units_MRT',
       'Units_OTR', 'Units_PASS', 'Units_PSUV', 'Units_RLT', 'Units_SPC',
       'Units_TRLR', 'Units_-', 'Units_Customer-Billed',
       'Units_National Acct'],
      dtype='object')
Index(['NumOfPrograms', 'BO_GROUP', 'CBI_GROUP', 'CDGTREAD', 'CDGVTP',
       'ICOOPMED', 'IFLEETDLR', 'IHANKONE', 'IMASTALLI', 'INEXNTLVL',
       'INITTOENT', 'INTWMT', 'INTWT3', 'IPIRFSTRK', 'ISIGNPRO', 'ITADMILLNM',
       'ITADSEPLUS', 'ITBCCORE', 'ITOYODRVN', 'OREXCEPT'],
      dtype='object')


In [16]:
data.describe()

,Customer_ID,recency,frequency,value,age,units,Units_BLT,Units_FRM,Units_IND,Units_MBT,...,INITTOENT,INTWMT,INTWT3,IPIRFSTRK,ISIGNPRO,ITADMILLNM,ITADSEPLUS,ITBCCORE,ITOYODRVN,OREXCEPT
count,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.0,2000.000000,2000.000000,2000.000000,2000.0,2000.000000,2000.000000,2000.000000,2000.0
mean,3.060826e+07,48.858000,87.221000,480.995590,287.642000,280.374474,0.393928,1.503271,0.775262,0.132505,...,0.007000,0.0,0.013000,0.001000,0.003000,0.0,0.010500,0.036500,0.000500,0.0
std,1.038077e+07,85.829244,278.567101,575.517781,170.286255,1119.791115,2.815687,8.566144,5.050475,0.974669,...,0.083394,0.0,0.113302,0.031615,0.054704,0.0,0.101956,0.187578,0.022361,0.0
min,1.000011e+07,1.000000,1.000000,21.960000,1.000000,0.038593,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
25%,2.528443e+07,4.000000,4.750000,275.485630,147.000000,12.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
50%,3.407979e+07,12.000000,17.000000,384.518418,274.000000,48.749337,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
75%,4.002172e+07,49.000000,60.000000,530.049583,430.000000,173.129705,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
max,4.006232e+07,561.000000,7240.000000,13579.500000,605.000000,34485.496354,57.000000,154.000000,97.000000,18.000000,...,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0


In [17]:
x = data.drop('churn', axis=1)
y = data['churn']
print(x)
print(y)
(y==0).sum(), (y==1).sum()

      Customer_ID  recency  frequency        value    age        units  \
0        10083892     85.0        8.0   171.661250  316.0    14.000000   
1        40018099      2.0      469.0   441.328591  240.0  1516.476421   
2        33491280      4.0       18.0   826.507222  179.0    74.000000   
3        35878660      6.0      179.0   424.440056  391.0   419.000000   
4        34542857      1.0      223.0  1670.450451  575.0   729.750000   
...           ...      ...        ...          ...    ...          ...   
1995     34078678      7.0       37.0   247.412703  210.0    83.000000   
1996     40034609     10.0       24.0   438.747917   87.0    63.514563   
1997     40042684     26.0       45.0   684.805955  257.0   157.192661   
1998     40047272     11.0       23.0   426.842174  452.0    70.000000   
1999     33549389     22.0       29.0   664.741379  575.0    93.000000   

      Units_BLT  Units_FRM  Units_IND  Units_MBT  ...  INITTOENT  INTWMT  \
0           0.0        0.0        0

(np.int64(1638), np.int64(362))

In [18]:
initial_selector = SelectKBest(score_func=f_classif, k='all')
initial_selector.fit(x, y)
initial_importance = pd.DataFrame({
    'feature': x.columns,
    'f_score': initial_selector.scores_,
    'p_value': initial_selector.pvalues_
})


initial_importance_sorted = initial_importance.sort_values('f_score', ascending=False)

plt.figure(figsize=(15, 8))
sns.barplot(x='f_score', y='feature', data=initial_importance_sorted.head(30))
plt.title('Feature Importance Scores (Top 30 Features)')
plt.xlabel('F-score')
plt.tight_layout()
plt.show()

print("\nTop 30 features by F-score:")
print(initial_importance_sorted.head(20))

top_20_by_pvalue = initial_importance.sort_values('p_value').head(20)['feature'].tolist()
print("\nTop 20 features by lowest p-value:")
print(top_20_by_pvalue)


d:\AI COE Trainings\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [ 30  33  34  35  43  44  48  60  68  75  76  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97 128 131 132 133 141 142
 146 158 166 173 174 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 219 223 227] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
d:\AI COE Trainings\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



Top 30 features by F-score:
                    feature     f_score        p_value
1                   recency  750.373404  1.552880e-140
202                   Units   57.808962   4.415625e-14
201            Sales_Amount   55.309066   1.520255e-13
205              Channel_NR   36.025645   2.306620e-09
40            Units_HANKOOK   32.934341   1.099040e-08
138    Sales_Amount_HANKOOK   32.237401   1.564028e-08
2                 frequency   32.180842   1.609481e-08
207               IsProgram   29.450052   6.434889e-08
98                    sales   27.221899   2.001808e-07
196  Sales_Amount_is_win_NO   27.132927   2.094787e-07
111       Sales_Amount_PSUV   25.946211   3.840745e-07
198    Sales_Amount_is_ev_-   25.459330   4.927060e-07
199   Sales_Amount_is_ev_NO   23.605335   1.274641e-06
102          Units_is_ev_NO   21.894623   3.073553e-06
144   Sales_Amount_MICHELIN   21.873553   3.107115e-06
46           Units_MICHELIN   21.702248   3.394017e-06
5                     units   20.875

In [19]:
selected_features = top_20_by_pvalue
X_selected = data[selected_features]
print("Shape of selected features dataframe:", X_selected.shape)
print("\nSelected features:")
for feature in selected_features:
    print(feature)

Shape of selected features dataframe: (2000, 20)

Selected features:
recency
Units
Sales_Amount
Channel_NR
Units_HANKOOK
Sales_Amount_HANKOOK
frequency
IsProgram
sales
Sales_Amount_is_win_NO
Sales_Amount_PSUV
Sales_Amount_is_ev_-
Sales_Amount_is_ev_NO
Units_is_ev_NO
Sales_Amount_MICHELIN
Units_MICHELIN
units
NumOfPrograms
Units_is_win_NO
Units_PSUV


In [20]:
scaler = StandardScaler()
X_selected_scaled = scaler.fit_transform(X_selected)

X_selected_scaled = pd.DataFrame(X_selected_scaled, columns=selected_features)

print("Shape of scaled features:", X_selected_scaled.shape)
print("\nFirst few rows of scaled data:")
print(X_selected_scaled.head())

Shape of scaled features: (2000, 20)

First few rows of scaled data:
    recency     Units  Sales_Amount  Channel_NR  Units_HANKOOK  \
0  0.421197 -0.356705     -0.349119   -0.458172      -0.243919   
1 -0.546081  0.565343      0.509025    2.182588       0.758865   
2 -0.522773 -0.304281     -0.262853   -0.458172       0.224047   
3 -0.499465  0.032727      0.159859   -0.458172      -0.243919   
4 -0.557735 -0.053398      0.566200   -0.458172      -0.310771   

   Sales_Amount_HANKOOK  frequency  IsProgram     sales  \
0             -0.268419  -0.284459  -0.305767 -0.276407   
1              0.576717   1.370853  -0.305767  1.095056   
2              0.363382  -0.248552  -0.305767 -0.186333   
3             -0.217213   0.329551   3.270460  0.221202   
4             -0.306795   0.487541   3.270460  2.199160   

   Sales_Amount_is_win_NO  Sales_Amount_PSUV  Sales_Amount_is_ev_-  \
0               -0.275963          -0.272662             -0.270815   
1                1.096294           1.2

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [22]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("Model Performance Metrics:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"R^2 (score): {r2:.3f}")
print(f"Mean Squared Error: {mse:.3f}")
print("\nConfusion Matrix:")
print(conf_matrix)

Model Performance Metrics:
Accuracy: 0.835
Precision: 0.784
Recall: 0.333
R^2 (score): 0.031
Mean Squared Error: 0.165

Confusion Matrix:
[[305   8]
 [ 58  29]]
